In [1]:
import pandas as pd

In [2]:
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')

In [3]:
melb_df = melb_data.copy()

# ФОРМАТ DATETIME

Таким форматом в Pandas является формат **datetime**, который записывается как YYYY-MM-DD HH: MM: SS, то есть составляющие времени указываются в следующем порядке: год, месяц, день, час, минута, секунда.

В наших данных дата записана в виде DD/MM/YYYY, например 3/12/2017. Посмотрим на это:

In [4]:
display(melb_df['Date'])

0         3/12/2016
1         4/02/2016
2         4/03/2017
3         4/03/2017
4         4/06/2016
            ...    
13575    26/08/2017
13576    26/08/2017
13577    26/08/2017
13578    26/08/2017
13579    26/08/2017
Name: Date, Length: 13580, dtype: object

Для того чтобы преобразовывать столбцы с датами, записанными в распространённых форматах, в формат datetime, можно воспользоваться функцией pandas.to_datetime(). В нашем случае в функции нужно указать параметр dayfirst=True, который будет обозначать, что в первоначальном признаке первым идет день. Преобразуем столбец Date в формат datetime, передав его в эту функцию:

In [5]:
melb_df['Date'] = pd.to_datetime(melb_df['Date'], dayfirst=True)
display(melb_df['Date'])

0       2016-12-03
1       2016-02-04
2       2017-03-04
3       2017-03-04
4       2016-06-04
           ...    
13575   2017-08-26
13576   2017-08-26
13577   2017-08-26
13578   2017-08-26
13579   2017-08-26
Name: Date, Length: 13580, dtype: datetime64[ns]

В результате мы переопределяем признак Date в формат datetime. При этом так как в изначальном варианте время не было указано, то и после преобразования оно опускается.

Стоит обратить внимание, что изменился тип данных для столбца Date, теперь его тип — datetime64.

# ВЫДЕЛЕНИЕ АТРИБУТОВ DATETIME

Тип данных datetime позволяет с помощью специального аксессора dt выделять составляющие времени из каждого элемента столбца, такие как:

* **date** — дата;
* **year, month, day** — год, месяц, день;
* **time** — время;
* **hour, minute, second** — час, минута, секунда;
* **dayofweek** — номер дня недели, от 0 до 6, где 0 — понедельник, 6 — воскресенье;
* **day_name** — название дня недели;
* **dayofyear** — порядковый день года;
* **quarter** — квартал (интервал в три месяца).

Например, обратившись по атрибуту dt.year в столбце Date, мы можем «достать» год продажи и понять, за какой интервал времени (в годах) представлены наши данные, а также на какой год приходится наибольшее число продаж:

In [8]:
years_sold = melb_df['Date'].dt.year
print(years_sold)
print('Min year sold:', years_sold.min())
print('Max year sold:', years_sold.max())
print('Mode year sold:', years_sold.mode()[0])

0        2016
1        2016
2        2017
3        2017
4        2016
         ... 
13575    2017
13576    2017
13577    2017
13578    2017
13579    2017
Name: Date, Length: 13580, dtype: int64
Min year sold: 2016
Max year sold: 2017
Mode year sold: 2017


**Примечание.** Так как модальных значений в столбце может быть несколько, метод mode() возвращает объект Series, даже если мода в данных только одна. Чтобы сохранить стилистику вывода информации о годе продажи и выводить только число, а не Series, мы обращаемся к результату работы метода mode() по индексу 0.

Теперь попробуем понять, на какие месяцы приходится пик продаж объектов недвижимости. Для этого выделим атрибут dt.month и на этот раз занесём результат в столбец MonthSale, а затем найдём относительную частоту продаж для каждого месяца от общего количества продаж — для этого используем метод value_counts() с параметром normalize (вывод в долях):

In [9]:
melb_df['MonthSale'] = melb_df['Date'].dt.month
melb_df['MonthSale'].value_counts(normalize=True)

5     0.149411
7     0.145950
9     0.135862
6     0.134757
8     0.114138
11    0.082032
4     0.069882
3     0.049926
12    0.044698
10    0.040574
2     0.032622
1     0.000147
Name: MonthSale, dtype: float64

Из результатов становится ясно, что наибольшее количество продаж недвижимости приходится на май, июль и сентябрь (пятый, седьмой и девятый месяцы соответственно). Месяцами застоя при этом являются месяцы — октябрь, февраль и январь (десятый, второй и первый месяцы соответственно).

# РАБОТА С ИНТЕРВАЛАМИ

Часто бывает такая ситуация, что необходимо вычислять интервалы между двумя временными промежутками. Например, можно вычислить, сколько дней прошло с 1 января 2016 года до момента продажи объекта. Для этого можно просто найти разницу между датами продаж и заявленной датой, представленной в формате datetime:

In [10]:
delta_days = melb_df['Date'] - pd.to_datetime('2016-01-01') 
display(delta_days)

0       337 days
1        34 days
2       428 days
3       428 days
4       155 days
          ...   
13575   603 days
13576   603 days
13577   603 days
13578   603 days
13579   603 days
Name: Date, Length: 13580, dtype: timedelta64[ns]

В результате мы получаем Series, элементами которой является количество дней, которое прошло с 1 января 2016 года. Обратите внимание, что данные такого формата относятся к типу timedelta.

Чтобы превратить количество дней из формата интервала в формат целого числа дней, можно воспользоваться аксессором dt для формата timedelta и извлечь из него атрибут days:

In [11]:
display(delta_days.dt.days)

0        337
1         34
2        428
3        428
4        155
        ... 
13575    603
13576    603
13577    603
13578    603
13579    603
Name: Date, Length: 13580, dtype: int64

Рассмотрим другой пример. Давайте создадим признак возраста объекта недвижимости в годах на момент продажи. Для этого выделим из столбца с датой продажи год и вычтем из него год постройки здания. Результат оформим в виде столбца AgeBuilding:

In [12]:
melb_df['AgeBuilding'] = melb_df['Date'].dt.year - melb_df['YearBuilt']
display(melb_df['AgeBuilding'])

0         46
1        116
2        117
3         47
4          2
        ... 
13575     36
13576     22
13577     20
13578     97
13579     97
Name: AgeBuilding, Length: 13580, dtype: int64

**Примечание.** Обратите внимание, что, так как года кодируются целым числом, в результате мы тоже получаем целочисленный столбец — тип int64 (а не timedelta).

На самом деле столбец AgeBuilding дублирует информацию столбца YearBuilt, так как, зная год постройки здания, мы автоматически знаем его возраст. Такие признаки не стоит оставлять вместе, поэтому оставим возраст здания, так как он является более наглядным, а год постройки удалим из таблицы:

In [13]:
melb_df = melb_df.drop('YearBuilt', axis=1)

# Практика

3.3. Создайте в таблице melb_df признак WeekdaySale (день недели). Найдите, сколько объектов недвижимости было продано в выходные (суббота и воскресенье), результат занесите в переменную weekend_count. В качестве ответа введите результат вывода переменной weekend_count.

In [19]:
melb_df['WeekdaySale'] = melb_df['Date'].dt.dayofweek
weekend_count = melb_df[(melb_df['WeekdaySale'] == 5) | (melb_df['WeekdaySale'] == 6)].shape[0]
print(f"sales on weekends = {weekend_count}")

sales on weekends = 12822


ВВОДНЫЕ ДАННЫЕ ДЛЯ ВЫПОЛНЕНИЯ ЗАДАНИЙ 3.4-3.5

Вам представлены данные (в формате csv) об отчётах очевидцев НЛО в США за период с 1930 по 2020 год.

В данных есть следующие признаки:

* **"City"** — город, где был замечен НЛО;
* **"Colors Reported"** — цвет объекта;
* **"Shape Reported"** — форма объекта;
* **"State"** — обозначение штата;
* **"Time"** — время, когда был замечен НЛО (данные отсортированы от старых наблюдений к новым).
 
Прочитайте данные, сделайте преобразование времени к формату datetime и выполните задания ниже.

In [20]:
import requests
import io

In [21]:
urlData = requests.get('https://raw.githubusercontent.com/justmarkham/pandas-videos/master/data/ufo.csv').content

ufo_state = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
display(ufo_state)

,City,Colors Reported,Shape Reported,State,Time
0,Ithaca,NaN,TRIANGLE,NY,6/1/1930 22:00
1,Willingboro,NaN,OTHER,NJ,6/30/1930 20:00
2,Holyoke,NaN,OVAL,CO,2/15/1931 14:00
3,Abilene,NaN,DISK,KS,6/1/1931 13:00
4,New York Worlds Fair,NaN,LIGHT,NY,4/18/1933 19:00
...,...,...,...,...,...
18236,Grant Park,NaN,TRIANGLE,IL,12/31/2000 23:00
18237,Spirit Lake,NaN,DISK,IA,12/31/2000 23:00
18238,Eagle River,NaN,NaN,WI,12/31/2000 23:45
18239,Eagle River,RED,LIGHT,WI,12/31/2000 23:45


In [24]:
ufo_state_dp = ufo_state.copy()

In [25]:
ufo_state_dp['Time'] = pd.to_datetime(ufo_state_dp['Time'], dayfirst=True)
display(ufo_state_dp['Time'])

0       1930-01-06 22:00:00
1       1930-06-30 20:00:00
2       1931-02-15 14:00:00
3       1931-01-06 13:00:00
4       1933-04-18 19:00:00
                ...        
18236   2000-12-31 23:00:00
18237   2000-12-31 23:00:00
18238   2000-12-31 23:45:00
18239   2000-12-31 23:45:00
18240   2000-12-31 23:59:00
Name: Time, Length: 18241, dtype: datetime64[ns]

3.4. В каком году отмечается наибольшее количество случаев наблюдения НЛО в США?

In [29]:
ufo_state_dp['Year'] = ufo_state_dp['Time'].dt.year
ufo_state_dp.Year.value_counts()

1999    2774
2000    2635
1998    1743
1995    1344
1997    1237
        ... 
1936       2
1930       2
1935       1
1934       1
1933       1
Name: Year, Length: 68, dtype: int64

3.5. Найдите средний интервал времени (в днях) между двумя последовательными случаями наблюдения НЛО в штате Невада (NV).

Чтобы выделить дату из столбца Time, можно воспользоваться атрибутом datetime date.

Чтобы вычислить разницу между двумя соседними датами в столбце, примените к нему метод diff().

Чтобы перевести интервал времени в дни, воспользуйтесь атрибутом timedelta days.

In [47]:
dates = ufo_state_dp[ufo_state_dp['State'] == 'NV'].Time.dt.date
diff_dates = dates.diff()
diff_dates.dt.days.mean()

68.72084805653711